In [9]:
import os
import numpy as np
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Initialize the pre-trained MobileNet model without the top classification layer
model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3), pooling='avg')

# Function to extract features from an image
def extract_features(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)  # Preprocess for MobileNet
    features = model.predict(img_array)
    return features.flatten()

# Directory paths
apple_pie_train_dir = '../data/food11/train/apple_pie'
apple_pie_test_dir = '../data/food11/test/apple_pie'

# Collect all image paths in the pizza folders
image_paths = [os.path.join(apple_pie_train_dir, fname) for fname in os.listdir(apple_pie_train_dir)]
image_paths += [os.path.join(apple_pie_test_dir, fname) for fname in os.listdir(apple_pie_test_dir)]

# Extract features for all images
features = []
for img_path in image_paths:
    try:
        features.append(extract_features(img_path, model))
    except Exception as e:
        print(f"Error processing image {img_path}: {e}")

features = np.array(features)
print(f"Extracted features for {len(features)} images")

Extracted features for 1000 images


In [10]:
from sklearn.cluster import KMeans

# Number of clusters (choose based on expected subcategories, e.g., 3 types of pizza)
num_clusters = 3

# Perform K-Means clustering
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(features)

# Get cluster labels for each image
cluster_labels = kmeans.labels_

# Print the cluster label for each image
for img_path, label in zip(image_paths, cluster_labels):
    print(f"{img_path} assigned to cluster {label}")

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


../data/food11/train/apple_pie\1005649.jpg assigned to cluster 2
../data/food11/train/apple_pie\1011328.jpg assigned to cluster 1
../data/food11/train/apple_pie\1014775.jpg assigned to cluster 2
../data/food11/train/apple_pie\1026328.jpg assigned to cluster 2
../data/food11/train/apple_pie\1028787.jpg assigned to cluster 2
../data/food11/train/apple_pie\1034399.jpg assigned to cluster 1
../data/food11/train/apple_pie\1038694.jpg assigned to cluster 1
../data/food11/train/apple_pie\1043283.jpg assigned to cluster 2
../data/food11/train/apple_pie\1047447.jpg assigned to cluster 1
../data/food11/train/apple_pie\1050519.jpg assigned to cluster 2
../data/food11/train/apple_pie\1057749.jpg assigned to cluster 1
../data/food11/train/apple_pie\1057810.jpg assigned to cluster 0
../data/food11/train/apple_pie\1068632.jpg assigned to cluster 2
../data/food11/train/apple_pie\1072416.jpg assigned to cluster 0
../data/food11/train/apple_pie\1074856.jpg assigned to cluster 2
../data/food11/train/appl

In [11]:
import shutil

# Create directories for each cluster (subcategories)
for i in range(num_clusters):
    train_subdir = os.path.join(apple_pie_train_dir, f"cluster_{i}")
    test_subdir = os.path.join(apple_pie_test_dir, f"cluster_{i}")
    os.makedirs(train_subdir, exist_ok=True)
    os.makedirs(test_subdir, exist_ok=True)

# Move images to subdirectories based on cluster labels
for img_path, label in zip(image_paths, cluster_labels):
    if 'train' in img_path:
        shutil.move(img_path, os.path.join(apple_pie_train_dir, f"cluster_{label}", os.path.basename(img_path)))
    elif 'test' in img_path:
        shutil.move(img_path, os.path.join(apple_pie_test_dir, f"cluster_{label}", os.path.basename(img_path)))

print("Images moved to their respective cluster directories.")

Images moved to their respective cluster directories.


In [16]:
# Fine-grained apple_pie classification
apple_pie_train_gen = ImageDataGenerator(rescale=1./255)
apple_pie_test_gen = ImageDataGenerator(rescale=1./255)

apple_pie_train_generator = apple_pie_train_gen.flow_from_directory(
    apple_pie_train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

apple_pie_test_generator = apple_pie_test_gen.flow_from_directory(
    apple_pie_test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Use the same base model (MobileNet)
base_model_apple_pie = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
for layer in base_model_apple_pie.layers:
    layer.trainable = False

# Add custom layers for apple_pie classification
x_apple_pie = base_model_apple_pie.output
x_apple_pie = GlobalAveragePooling2D()(x_apple_pie)
x_apple_pie = Dense(512, activation='relu')(x_apple_pie)
apple_pie_predictions = Dense(num_clusters, activation='softmax')(x_apple_pie)

# Create and compile the model
apple_pie_model = Model(inputs=base_model_apple_pie.input, outputs=apple_pie_predictions)
apple_pie_model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the fine-grained apple_pie classification model
apple_pie_model.fit(
    apple_pie_train_generator,
    steps_per_epoch=apple_pie_train_generator.samples // 32,
    validation_data=apple_pie_test_generator,
    validation_steps=apple_pie_test_generator.samples // 32,
    epochs=30
)

# Save the model
apple_pie_model.save('fine_grained_apple_pie_classifier.h5')

Found 900 images belonging to 3 classes.
Found 100 images belonging to 3 classes.
Epoch 1/30
28/28 [==============================] - 8s 276ms/step - loss: 0.8722 - accuracy: 0.6152 - val_loss: 0.3887 - val_accuracy: 0.8958
Epoch 2/30
28/28 [==============================] - 8s 271ms/step - loss: 0.3591 - accuracy: 0.8952 - val_loss: 0.2329 - val_accuracy: 0.9688
Epoch 3/30
28/28 [==============================] - 7s 262ms/step - loss: 0.2386 - accuracy: 0.9378 - val_loss: 0.1982 - val_accuracy: 0.9688
Epoch 4/30
28/28 [==============================] - 7s 256ms/step - loss: 0.1776 - accuracy: 0.9666 - val_loss: 0.1906 - val_accuracy: 0.9583
Epoch 5/30
28/28 [==============================] - 7s 256ms/step - loss: 0.1382 - accuracy: 0.9712 - val_loss: 0.1407 - val_accuracy: 0.9583
Epoch 6/30
28/28 [==============================] - 7s 262ms/step - loss: 0.1184 - accuracy: 0.9793 - val_loss: 0.1318 - val_accuracy: 0.9583
Epoch 7/30
28/28 [==============================] - 7s 260ms/step 